In [1]:
import copy
import json
import sqlite3
import pandas as pd
import datetime,time
import os

In [8]:
def report_query(param):
    ## 기본 쿼리 작성
    param_ = {}
    param_['days'] = param.get('days','60') ## 기본 60일
    param_['dpt'] = param['dpt'] ## 출도착지가 없으면 에러
    param_['arr'] = param['arr']
    scrap_date = param.get('scrap_date')
    if scrap_date is None:
        scrap_date = "(select max(scrap_date) from airfare_scraped_data where scrap_site in ({scrap_site}) and dpt='{dpt}' and arr='{arr}')"
    else:
        scrap_date = "'{scrap_date}'"
        param_['scrap_date'] = param['scrap_date']
    sql = '''select airline, flt, dpt_date, dpt, arr, dpt_time, arr_time, fare, tax1, tax2, seat
        from airfare_scraped_data
        where scrap_date = '''
    sql = sql + scrap_date
    sql = sql + ''' and scrap_site in ({scrap_site})
        and dpt='{dpt}' and arr='{arr}'
        and dpt_date <= strftime('%Y%m%d','now','+{days} day')
        and airline in ({airline})'''
    param_['scrap_site'] = ','.join(["'"+c+"'" for c in param.get('sitecodes',[])])
    param_['airline'] = ','.join(["'"+c+"'" for c in param.get('airline',[])])
    query = sql.format(**param_)
    print("query : ",query)
    return query

In [6]:
db = 'airfare_scraped_data.db'
site_code = { "code":"DailyReport","func":"daily_report_func","days":"21",
             "isairline":False,"auth":"002",
             "sitecodes":['IP'],"airline":['7C','LJ','TW','ZE'],
             "dpt":"GMP","arr":"CJU"}

In [3]:
conn = sqlite3.connect(db)
with conn:
    cur = conn.cursor()
    cur.execute('select max(scrap_date) from airfare_scraped_data')
    scrap_date = cur.fetchone()[0]
scrap_date

'2017063014'

In [27]:
param = copy.deepcopy(site_code);
param['scrap_date'] = scrap_date
query = report_query(param)
query = "select airline, dpt_date, dpt, arr, flt, dpt_time, fare from(" +\
    query +\
    ") order by airline,dpt_date"
report_list = []
conn = sqlite3.connect(db)
with conn:
    cur = conn.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    for row in rows:
        ## 포맷 : 항공사,날짜,출발지,도착지,편명,출발시간,판매최저값
        report_list.append(row)
report_list

query :  select airline, flt, dpt_date, dpt, arr, dpt_time, arr_time, fare, tax1, tax2, seat
        from airfare_scraped_data
        where scrap_date = '2017063014' and scrap_site in ('IP')
        and dpt='GMP' and arr='CJU'
        and dpt_date <= strftime('%Y%m%d','now','+21 day')
        and airline in ('7C','LJ','TW','ZE')


[('7C', '20170701', 'GMP', 'CJU', '151', '0625', 56900.0),
 ('7C', '20170701', 'GMP', 'CJU', '101', '0630', 56900.0),
 ('7C', '20170701', 'GMP', 'CJU', '171', '0645', 56900.0),
 ('7C', '20170701', 'GMP', 'CJU', '103', '0700', 61900.0),
 ('7C', '20170701', 'GMP', 'CJU', '105', '0750', 97700.0),
 ('7C', '20170701', 'GMP', 'CJU', '107', '0900', 97700.0),
 ('7C', '20170701', 'GMP', 'CJU', '173', '1040', 87900.0),
 ('7C', '20170701', 'GMP', 'CJU', '119', '1350', 51900.0),
 ('7C', '20170701', 'GMP', 'CJU', '175', '1355', 46900.0),
 ('7C', '20170701', 'GMP', 'CJU', '121', '1440', 51900.0),
 ('7C', '20170701', 'GMP', 'CJU', '123', '1545', 97700.0),
 ('7C', '20170701', 'GMP', 'CJU', '177', '1730', 36900.0),
 ('7C', '20170701', 'GMP', 'CJU', '127', '1815', 41900.0),
 ('7C', '20170701', 'GMP', 'CJU', '129', '1905', 31900.0),
 ('7C', '20170701', 'GMP', 'CJU', '133', '1935', 41900.0),
 ('7C', '20170701', 'GMP', 'CJU', '147', '2025', 36900.0),
 ('7C', '20170701', 'GMP', 'CJU', '151', '0625', 97700.0

In [28]:
columns = ['airline', 'dpt_date', 'dpt', 'arr', 'flt', 'dpt_time', 'fare']
df = pd.DataFrame(report_list,columns=columns)
df.head()

,airline,dpt_date,dpt,arr,flt,dpt_time,fare
0,7C,20170701,GMP,CJU,151,0625,56900
1,7C,20170701,GMP,CJU,101,0630,56900
2,7C,20170701,GMP,CJU,171,0645,56900
3,7C,20170701,GMP,CJU,103,0700,61900
4,7C,20170701,GMP,CJU,105,0750,97700


In [29]:
df['fare'].dtype

dtype('O')

In [30]:
df.ix[df['fare']=='',['fare']]

,fare
20,
21,
26,
101,
150,
162,
199,
282,
283,
297,


In [31]:
if df['fare'].dtype == object:
    df.ix[df['fare'] == '','fare'] = 0
df.ix[df['fare']=='']

,airline,dpt_date,dpt,arr,flt,dpt_time,fare


In [43]:
len([i for i in df['fare'].values if type(i) == int]),len(df['fare'])

(840, 3354)

In [24]:
def agg_min(arr):
    narr = arr.values
    narr = narr[narr > 0]
    if len(narr) <= 0:
        return 0
    else:
        return sorted(narr)[0]

In [33]:
dfmax = df.groupby(['airline','dpt_date','dpt','arr','flt','dpt_time']).max()
dfmax.head()

fare
airline dpt_date dpt arr flt dpt_time         
7C      20170701 GMP CJU 101 0630      97700.0
                         103 0700      97700.0
                         105 0750      97700.0
                         107 0900      97700.0
                         119 1350      97700.0

In [44]:
dfmin = df.groupby(['airline','dpt_date','dpt','arr','flt','dpt_time']).agg(agg_min)
dfmin = dfmin.reset_index()
dfmin.head()

,airline,dpt_date,dpt,arr,flt,dpt_time,fare
0,7C,20170701,GMP,CJU,101,0630,56900.0
1,7C,20170701,GMP,CJU,103,0700,61900.0
2,7C,20170701,GMP,CJU,105,0750,97700.0
3,7C,20170701,GMP,CJU,107,0900,97700.0
4,7C,20170701,GMP,CJU,119,1350,51900.0


In [45]:
df.ix[(df['dpt_date']=='20170701')&(df['flt']=='101')]

,airline,dpt_date,dpt,arr,flt,dpt_time,fare
1,7C,20170701,GMP,CJU,101,0630,56900
17,7C,20170701,GMP,CJU,101,0630,97700


In [46]:
dfze = dfmin[dfmin['airline'] == 'ZE']
dfze = dfze.reset_index()[dfmin.columns]
dfze.head()

,airline,dpt_date,dpt,arr,flt,dpt_time,fare
0,ZE,20170701,GMP,CJU,203,0650,59900.0
1,ZE,20170701,GMP,CJU,205,0810,90900.0
2,ZE,20170701,GMP,CJU,207,0950,0.0
3,ZE,20170701,GMP,CJU,209,1035,79900.0
4,ZE,20170701,GMP,CJU,213,1150,64900.0


In [47]:
dfmin['dpt_min'] = dfmin['dpt_time'].map(lambda x:int(x[:2])*60 + int(x[2:]))
dfmin['dpt_min'].head()

0    390
1    420
2    470
3    540
4    830
Name: dpt_min, dtype: int64

In [48]:
airlines = set(dfmin['airline'].unique()) - set(['ZE'])
dpt_dates = dfze['dpt_date'].unique()

In [49]:
airlines

{'7C', 'LJ', 'TW'}

In [50]:
dpt_dates

array(['20170701', '20170702', '20170703', '20170704', '20170705',
       '20170706', '20170707', '20170708', '20170709', '20170710',
       '20170711', '20170712', '20170713', '20170714', '20170715',
       '20170716', '20170717', '20170718', '20170719', '20170720',
       '20170721'], dtype=object)

In [51]:
cal_min = lambda x:int(x[:2])*60+int(x[2:])
## 값을 채울 컬럼 생성
for airline in  airlines:
    dfze[airline+'-info'] = ''
    dfze[airline+'-fare'] = 0
dfze.columns

Index(['airline', 'dpt_date', 'dpt', 'arr', 'flt', 'dpt_time', 'fare',
       '7C-info', '7C-fare', 'LJ-info', 'LJ-fare', 'TW-info', 'TW-fare'],
      dtype='object')

In [52]:
zeobj = dfze[dfze['dpt_date']=='20170703']
zeobj

,airline,dpt_date,dpt,arr,flt,dpt_time,fare,7C-info,7C-fare,LJ-info,LJ-fare,TW-info,TW-fare
33,ZE,20170703,GMP,CJU,203,0650,65500.0,,0,,0,,0
34,ZE,20170703,GMP,CJU,205,0810,0.0,,0,,0,,0
35,ZE,20170703,GMP,CJU,207,1010,65500.0,,0,,0,,0
36,ZE,20170703,GMP,CJU,209,1035,65500.0,,0,,0,,0
37,ZE,20170703,GMP,CJU,213,1150,65500.0,,0,,0,,0
38,ZE,20170703,GMP,CJU,215,1255,65500.0,,0,,0,,0
39,ZE,20170703,GMP,CJU,217,1435,65500.0,,0,,0,,0
40,ZE,20170703,GMP,CJU,219,1540,65500.0,,0,,0,,0
41,ZE,20170703,GMP,CJU,223,1635,65500.0,,0,,0,,0
42,ZE,20170703,GMP,CJU,225,1835,65500.0,,0,,0,,0


In [68]:
dfcmp = dfmin[(dfmin['airline'] == '7C') & (dfmin['dpt_date'] == '20170703')]
dfcmp

,airline,dpt_date,dpt,arr,flt,dpt_time,fare,dpt_min
39,7C,20170703,GMP,CJU,101,0630,56900.0,390
40,7C,20170703,GMP,CJU,103,0700,61900.0,420
41,7C,20170703,GMP,CJU,105,0750,61900.0,470
42,7C,20170703,GMP,CJU,113,1120,56900.0,680
43,7C,20170703,GMP,CJU,115,1225,51900.0,745
44,7C,20170703,GMP,CJU,117,1240,46900.0,760
45,7C,20170703,GMP,CJU,119,1350,41900.0,830
46,7C,20170703,GMP,CJU,121,1440,36900.0,880
47,7C,20170703,GMP,CJU,123,1545,41900.0,945
48,7C,20170703,GMP,CJU,125,1605,36900.0,965


In [72]:
cal_min = lambda x:int(x[:2])*60+int(x[2:])
zeobj_i = zeobj.index[0]
dfcmp['dpt_cmp'] = abs(dfcmp['dpt_min'] - cal_min(zeobj.ix[zeobj_i,'dpt_time']))
dfcmp

d:\python\python35\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,airline,dpt_date,dpt,arr,flt,dpt_time,fare,dpt_min,dpt_cmp
39,7C,20170703,GMP,CJU,101,0630,56900.0,390,20
40,7C,20170703,GMP,CJU,103,0700,61900.0,420,10
41,7C,20170703,GMP,CJU,105,0750,61900.0,470,60
42,7C,20170703,GMP,CJU,113,1120,56900.0,680,270
43,7C,20170703,GMP,CJU,115,1225,51900.0,745,335
44,7C,20170703,GMP,CJU,117,1240,46900.0,760,350
45,7C,20170703,GMP,CJU,119,1350,41900.0,830,420
46,7C,20170703,GMP,CJU,121,1440,36900.0,880,470
47,7C,20170703,GMP,CJU,123,1545,41900.0,945,535
48,7C,20170703,GMP,CJU,125,1605,36900.0,965,555


In [73]:
dfcmp['dpt_cmp'].argmin()

40

In [71]:
import math
abs(-2.8)

2.8